In [1]:
import findspark
findspark.init()
findspark.find()

'/home/henry/spark'

In [2]:
from pyspark.sql import SparkSession

In [3]:
from graphframes import GraphFrame

In [4]:
spark = SparkSession.builder.appName("GraphFrameBasis").getOrCreate()

22/09/26 23:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


- Graph is a pair (V, E)
- V is a set of vertices
- E is a collection of pair of vertices, called edge
- V & E are positions and store elements

## 1. GraphFrame Basic

In [5]:
vertices = [("a", "Alice", 34),
            ("b", "Bob", 36),
            ("c", "Charlie", 30)]
column_names = ["id", "name", "age"]

In [6]:
v = spark.createDataFrame(vertices, column_names)
v.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
+---+-------+---+



In [7]:
edges = [("a", "b", "friend"),
        ("b", "c", "follow"), 
        ("c", "b", "follow")]
edge_columns = ["src", "dst", "relationship"]
e = spark.createDataFrame(edges, edge_columns)

In [8]:
e.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
+---+---+------------+



In [9]:
# create GraphFrame
graph = GraphFrame(v, e)
graph

GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])

In [10]:
# show in-degree of vertex(num of edges that terminate in that vertiex)
graph.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  c|       1|
|  b|       2|
+---+--------+



In [11]:
# count number of follow connection in graph
graph.edges.filter("relationship = 'follow'").count()

2

In [12]:
# test PageRank algorithm
pageRanks = graph.pageRank(resetProbability=0.01, maxIter=20)
pageRanks.vertices.select("id", 'pagerank').show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  b|1.0905890109440908|
|  a|              0.01|
|  c|1.8994109890559092|
+---+------------------+



## 2. GraphFrame Algorithms

### 2.1 Create GraphFrame

In [13]:
vertices = [('a', 'Alice',34), 
            ('b', 'Bob', 36), 
            ('c', 'Charlie',30), 
            ('d', 'David',29), 
            ('e', 'Esther',32), 
            ('f', 'Fanny',36), 
            ('g', 'Gabby',60)]

edges = [('a', 'b', 'friend'),
        ('b', 'c', 'follow'), 
        ('c', 'b', 'follow'), 
        ('f', 'c', 'follow'), 
        ('e', 'f', 'follow'), 
        ('e', 'd', 'friend'), 
        ('d', 'a', 'friend'), 
        ('a', 'e', 'friend')]

In [14]:
v = spark.createDataFrame(vertices, ["id", "name", "age"])
e = spark.createDataFrame(edges, ["src", "dst", "relationship"])
user_graph = GraphFrame(v, e)

In [15]:
user_graph.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [16]:
user_graph.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



### 2.2 Triangles Finding

In [17]:
triangle_count = user_graph.triangleCount()
triangle_count.show() # count the number of triangle passing through each vertex

+-----+---+-------+---+
|count| id|   name|age|
+-----+---+-------+---+
|    0|  g|  Gabby| 60|
|    0|  f|  Fanny| 36|
|    1|  e| Esther| 32|
|    1|  d|  David| 29|
|    0|  c|Charlie| 30|
|    0|  b|    Bob| 36|
|    1|  a|  Alice| 34|
+-----+---+-------+---+



In [18]:
triangle_count.select("id", "count").show()

+---+-----+
| id|count|
+---+-----+
|  g|    0|
|  f|    0|
|  e|    1|
|  d|    1|
|  c|    0|
|  b|    0|
|  a|    1|
+---+-----+



In [19]:
triangle_count.select("id", "count").filter("count > 0").show()

+---+-----+
| id|count|
+---+-----+
|  e|    1|
|  d|    1|
|  a|    1|
+---+-----+



### 2.3 Motif Finding
- GraphFrames uses a simple domain-specific language (DSL) for expressing structural querie

In [20]:
# search for pairs of vertices {a, b} connected by edges in both directions
user_graph.find("(a)-[e1]->(b); (b)-[e2]->(a)").show()

+----------------+--------------+----------------+--------------+
|               a|            e1|               b|            e2|
+----------------+--------------+----------------+--------------+
|[c, Charlie, 30]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|
|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 30]|[c, b, follow]|
+----------------+--------------+----------------+--------------+



In [21]:
# search for pairs of vertices {a, b} has only one way relationship
user_graph.find("(a)-[e1]->(b); !(b)-[]->(a)").show()

+---------------+--------------+----------------+
|              a|            e1|               b|
+---------------+--------------+----------------+
| [a, Alice, 34]|[a, e, friend]| [e, Esther, 32]|
|[e, Esther, 32]|[e, f, follow]|  [f, Fanny, 36]|
| [a, Alice, 34]|[a, b, friend]|    [b, Bob, 36]|
|[e, Esther, 32]|[e, d, friend]|  [d, David, 29]|
| [f, Fanny, 36]|[f, c, follow]|[c, Charlie, 30]|
| [d, David, 29]|[d, a, friend]|  [a, Alice, 34]|
+---------------+--------------+----------------+



### 2.4 Finding unique triangles with motif

In [22]:
# triangle counting with motif finding
user_graph.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(a)").show()

+---------------+--------------+---------------+--------------+---------------+--------------+
|              a|            e1|              b|            e2|              c|            e3|
+---------------+--------------+---------------+--------------+---------------+--------------+
| [d, David, 29]|[d, a, friend]| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, d, friend]|
| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, d, friend]| [d, David, 29]|[d, a, friend]|
|[e, Esther, 32]|[e, d, friend]| [d, David, 29]|[d, a, friend]| [a, Alice, 34]|[a, e, friend]|
+---------------+--------------+---------------+--------------+---------------+--------------+



In [23]:
!head -4 data/sample_graph_vertices.txt

vertex_id
0
1
2


In [24]:
!head -4 data/sample_graph_edges.txt

edge_weight,from_id,to_id
0,5,15
1,18,8
2,6,1


In [25]:
vertice_df = spark.read.csv('data/sample_graph_vertices.txt', header=True)
vertice_df = vertice_df.withColumnRenamed("vertex_id", "id")
vertice_df.show(5)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [26]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [27]:
edges_df = spark.read.csv('data/sample_graph_edges.txt',header=True)
edges_df = edges_df.toDF("weight", "src", "dst").select("src", "dst")
edges_df.show(5)

+---+---+
|src|dst|
+---+---+
|  5| 15|
| 18|  8|
|  6|  1|
|  0| 10|
|  2|  4|
+---+---+
only showing top 5 rows



In [28]:
sample_graph = GraphFrame(vertice_df, edges_df)

In [29]:
# find all triangles, might have dupplicates
motifs = graph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")
motifs.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
+---+---+---+



In [30]:
# there only one change this case happen
unique_triangles = motifs[(motifs.a > motifs.b) & (motifs.b > motifs.c)]
unique_triangles.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
+---+---+---+



### 2.5 Finding Subgraph

In [31]:
user_paths = user_graph.find("(a)-[e]->(b)").filter("e.relationship = 'follow'"
                                                   ).filter("a.age < b.age")
selected_edges = user_paths.select("e.src", "e.dst", "e.relationship")
sample_subGraph = GraphFrame(user_graph.vertices, selected_edges)

In [32]:
sample_subGraph.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [33]:
sample_subGraph.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  e|  f|      follow|
|  c|  b|      follow|
+---+---+------------+



### 2.6 Friend Recommendation

In [34]:
potential_graph = user_graph.find("(A)-[]->(B); (B)-[]->(C); !(A)-[]->(C)")
filtered_graph = potential_graph.filter("A.id != C.id")
recommendation = filtered_graph.select("A", "C")
recommendation.show()

+---------------+----------------+
|              A|               C|
+---------------+----------------+
|[e, Esther, 32]|[c, Charlie, 30]|
|[e, Esther, 32]|  [a, Alice, 34]|
| [d, David, 29]|    [b, Bob, 36]|
| [a, Alice, 34]|  [d, David, 29]|
| [f, Fanny, 36]|    [b, Bob, 36]|
| [d, David, 29]| [e, Esther, 32]|
| [a, Alice, 34]|  [f, Fanny, 36]|
| [a, Alice, 34]|[c, Charlie, 30]|
+---------------+----------------+



## 3. Real World Applications 

### 3.1 Flight Data

In [35]:
airports_path = 'data/airports.json'
air_port_vertices = spark.read.json(airports_path)
air_port_vertices.show()

+-------------+-------+-----+---+
|         City|Country|State| id|
+-------------+-------+-----+---+
|      Chicago|    USA|   IL|ORD|
|     New York|    USA|   NY|LGA|
|       Boston|    USA|   MA|BOS|
|      Houston|    USA|   TX|IAH|
|       Newark|    USA|   NJ|EWR|
|       Denver|    USA|   CO|DEN|
|        Miami|    USA|   FL|MIA|
|San Francisco|    USA|   CA|SFO|
|      Atlanta|    USA|   GA|ATL|
|       Dallas|    USA|   TX|DFW|
|    Charlotte|    USA|   NC|CLT|
|  Los Angeles|    USA|   CA|LAX|
|      Seattle|    USA|   WA|SEA|
+-------------+-------+-----+---+



In [36]:
flight_path = 'data/flightdata2018.json'
flight_edges = spark.read.json(flight_path).select("src", "dst", "dist", "depdelay")
flight_edges.show()

+---+---+-----+--------+
|src|dst| dist|depdelay|
+---+---+-----+--------+
|ATL|BOS|946.0|     0.0|
|ATL|BOS|946.0|     8.0|
|ATL|BOS|946.0|     9.0|
|ATL|BOS|946.0|     0.0|
|ATL|BOS|946.0|     6.0|
|ATL|BOS|946.0|     0.0|
|ATL|BOS|946.0|     0.0|
|ATL|BOS|946.0|    21.0|
|ATL|BOS|946.0|   198.0|
|ATL|BOS|946.0|    14.0|
|ATL|BOS|946.0|   215.0|
|ATL|CLT|226.0|     0.0|
|ATL|CLT|226.0|     0.0|
|ATL|CLT|226.0|     0.0|
|ATL|CLT|226.0|     0.0|
|ATL|CLT|226.0|     0.0|
|ATL|CLT|226.0|     0.0|
|ATL|CLT|226.0|    11.0|
|ATL|CLT|226.0|     1.0|
|ATL|CLT|226.0|     0.0|
+---+---+-----+--------+
only showing top 20 rows



In [37]:
flight_graph = GraphFrame(air_port_vertices, flight_edges)
flight_graph

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 2 more fields])

In [38]:
flight_graph.vertices.count()

13

In [39]:
flight_graph.edges.count()

14016

In [40]:
from pyspark.sql.functions import col

In [41]:
# which flight routes have the longest distance
flight_graph.edges.groupBy("src", "dst").max("dist"
                                            ).sort(col("max(dist)").desc()
                                                  ).show()

+---+---+---------+
|src|dst|max(dist)|
+---+---+---------+
|MIA|SEA|   2724.0|
|SEA|MIA|   2724.0|
|BOS|SFO|   2704.0|
|SFO|BOS|   2704.0|
|BOS|LAX|   2611.0|
|LAX|BOS|   2611.0|
|SFO|MIA|   2585.0|
|MIA|SFO|   2585.0|
|SFO|EWR|   2565.0|
|EWR|SFO|   2565.0|
|BOS|SEA|   2496.0|
|SEA|BOS|   2496.0|
|LAX|EWR|   2454.0|
|EWR|LAX|   2454.0|
|SEA|EWR|   2402.0|
|EWR|SEA|   2402.0|
|LAX|MIA|   2342.0|
|MIA|LAX|   2342.0|
|SFO|CLT|   2296.0|
|CLT|SFO|   2296.0|
+---+---+---------+
only showing top 20 rows



In [42]:
# which flight routes have the highest average delays
flight_graph.edges.groupBy("src", "dst"
                          ).avg("depdelay"
                               ).sort(col("avg(depdelay)").desc()
                                     ).show()

+---+---+------------------+
|src|dst|     avg(depdelay)|
+---+---+------------------+
|SEA|MIA| 49.06666666666667|
|BOS|SEA|              36.0|
|EWR|LAX| 34.96739130434783|
|SEA|EWR| 34.31818181818182|
|MIA|SEA|33.142857142857146|
|MIA|EWR| 32.84415584415584|
|DEN|EWR|            31.875|
|BOS|EWR|31.463768115942027|
|EWR|DEN|30.430379746835442|
|LGA|CLT|            30.225|
|EWR|MIA|29.810126582278482|
|ORD|LGA|28.375757575757575|
|EWR|ATL| 27.39855072463768|
|BOS|SFO| 26.93220338983051|
|EWR|BOS| 26.92753623188406|
|MIA|LGA|25.964285714285715|
|ATL|LGA|25.852459016393443|
|LGA|BOS| 24.88888888888889|
|EWR|ORD|24.761467889908257|
|BOS|LGA|24.164556962025316|
+---+---+------------------+
only showing top 20 rows



In [43]:
# What are the longest delays for flights that are greater than 1,500 miles in distance?
# Note that the output here has been trimmed to fit the page.
flight_graph.edges.filter("dist > 1500"
                         ).orderBy(col("depdelay").desc()
                                  ).show(3)

+---+---+------+--------+
|src|dst|  dist|depdelay|
+---+---+------+--------+
|LAX|ATL|1947.0|   942.0|
|LAX|EWR|2454.0|   718.0|
|EWR|LAX|2454.0|   709.0|
+---+---+------+--------+
only showing top 3 rows



In [44]:
flight_graph.edges.show(3)

+---+---+-----+--------+
|src|dst| dist|depdelay|
+---+---+-----+--------+
|ATL|BOS|946.0|     0.0|
|ATL|BOS|946.0|     8.0|
|ATL|BOS|946.0|     9.0|
+---+---+-----+--------+
only showing top 3 rows



In [45]:
# What is the average delay for delayed flights departing from Atlanta (ATL)
flight_graph.edges.filter("src = 'ATL' and depdelay > 1"
                         ).groupBy("src", "dst"
                                  ).avg("depdelay"
                                       ).sort(col("avg(depdelay)").desc()
                                             ).show(3)

+---+---+------------------+
|src|dst|     avg(depdelay)|
+---+---+------------------+
|ATL|LGA|50.806451612903224|
|ATL|BOS|47.056603773584904|
|ATL|EWR|  46.9264705882353|
+---+---+------------------+
only showing top 3 rows



In [46]:
#What are the four most frequent flight routes in the dataset
flight_graph.edges.groupBy("src", "dst"
                          ).count().orderBy(col("count").desc()
                                           ).show(3)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|LAX|SFO|  201|
|ORD|LAX|  197|
|LAX|ORD|  196|
+---+---+-----+
only showing top 3 rows



In [47]:
# which airports have the most incomming and outgoing flights
flight_graph.degrees.orderBy(col("degree").desc()
                            ).show(3)

+---+------+
| id|degree|
+---+------+
|ORD|  3262|
|ATL|  2952|
|LAX|  2689|
+---+------+
only showing top 3 rows



In [48]:
# What are the most important airports, according to the PageRank algorithm
airport_ranks = flight_graph.pageRank(resetProbability=0.15, tol=0.01)
airport_ranks

GraphFrame(v:[id: string, City: string ... 3 more fields], e:[src: string, dst: string ... 3 more fields])

In [49]:
airport_ranks.vertices.orderBy(col("pageRank").desc()
                              ).show(5)

+-----------+-------+-----+---+------------------+
|       City|Country|State| id|          pagerank|
+-----------+-------+-----+---+------------------+
|    Chicago|    USA|   IL|ORD|1.4424205054493797|
|    Atlanta|    USA|   GA|ATL| 1.312213576108763|
|Los Angeles|    USA|   CA|LAX|1.2026295571856727|
|     Dallas|    USA|   TX|DFW|1.1671053525707842|
|     Denver|    USA|   CO|DEN|1.0888901678157008|
+-----------+-------+-----+---+------------------+
only showing top 5 rows



In [50]:
# fin all airport has not direct flight
flight_graph.find("(a)-[]->(b); (b)-[]->(c)").show(5)

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|[Atlanta, USA, GA...|[Boston, USA, MA,...|[San Francisco, U...|
|[Atlanta, USA, GA...|[Boston, USA, MA,...|[San Francisco, U...|
|[Atlanta, USA, GA...|[Boston, USA, MA,...|[San Francisco, U...|
|[Atlanta, USA, GA...|[Boston, USA, MA,...|[San Francisco, U...|
|[Atlanta, USA, GA...|[Boston, USA, MA,...|[San Francisco, U...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [51]:
# there is no direct path to connect 2 cities
flight_graph.bfs("id = 'LAX'", "id = 'LGA'", maxPathLength=1).show(5)

+----+-------+-----+---+
|City|Country|State| id|
+----+-------+-----+---+
+----+-------+-----+---+



In [52]:
# change value to 2 to get short test path connect 2 cities
flight_graph.bfs("id = 'LAX'", "id = 'LGA'", maxPathLength=2).show(5, truncate=False)

+---------------------------+-----------------------+-----------------------+------------------------+------------------------+
|from                       |e0                     |v1                     |e1                      |to                      |
+---------------------------+-----------------------+-----------------------+------------------------+------------------------+
|[Los Angeles, USA, CA, LAX]|[LAX, IAH, 1379.0, 0.0]|[Houston, USA, TX, IAH]|[IAH, LGA, 1416.0, 0.0] |[New York, USA, NY, LGA]|
|[Los Angeles, USA, CA, LAX]|[LAX, IAH, 1379.0, 0.0]|[Houston, USA, TX, IAH]|[IAH, LGA, 1416.0, 52.0]|[New York, USA, NY, LGA]|
|[Los Angeles, USA, CA, LAX]|[LAX, IAH, 1379.0, 0.0]|[Houston, USA, TX, IAH]|[IAH, LGA, 1416.0, 8.0] |[New York, USA, NY, LGA]|
|[Los Angeles, USA, CA, LAX]|[LAX, IAH, 1379.0, 0.0]|[Houston, USA, TX, IAH]|[IAH, LGA, 1416.0, 0.0] |[New York, USA, NY, LGA]|
|[Los Angeles, USA, CA, LAX]|[LAX, IAH, 1379.0, 0.0]|[Houston, USA, TX, IAH]|[IAH, LGA, 1416.0, 0.0] |[N

In [53]:
# we also can use motif finding to get all possible transit flight
flight_graph.find("(a)-[]->(b); (b)-[]->(c)"
                 ).filter("a.id = 'LAX'"
                         ).filter("c.id = 'LGA'"
                                 ).limit(5).select("a", "b", "c").show()

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|[Los Angeles, USA...|[Atlanta, USA, GA...|[New York, USA, N...|
|[Los Angeles, USA...|[Atlanta, USA, GA...|[New York, USA, N...|
|[Los Angeles, USA...|[Atlanta, USA, GA...|[New York, USA, N...|
|[Los Angeles, USA...|[Atlanta, USA, GA...|[New York, USA, N...|
|[Los Angeles, USA...|[Atlanta, USA, GA...|[New York, USA, N...|
+--------------------+--------------------+--------------------+

